<a href="https://colab.research.google.com/github/HausReport/ClubRaiders/blob/master/notebooks/ExpandRetreat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import math
import pandas as pd
import plotly.graph_objects as go
import humanize
import datetime as dt

In [129]:
def expandOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = 100.0 * (inf + top)/ bottom
  if ret> 100.0:
     ret = 100.0
  if ret<1.0:
     ret = 1.0
  return ret

def dailyIncrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [130]:
def retreatOneDay(inf: float, pop: int):
  top = (36 - math.log(pop,2))
  bottom = (100+ (36 - math.log(pop,2)))
  ret = inf - ((100 * ( (100-inf) + top)/ bottom) - (100-inf))
  if ret> 100.0:
     ret = 100.0
  if ret<1.0:
     ret = 1.0
  return ret

def dailyDecrease(inf: float, pop: int):
  return expandOneDay(inf,pop) - inf

In [131]:
retreatOneDay(9.2,2200)

7.3660866155447415

In [132]:
expandOneDay(65.9,2200)

72.6974398271657

In [133]:
def daysToPendingExpansion(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf < 75.0:
    day += 1
    tinf = expandOneDay(tinf, pop)
  return day

def daysToPendingRetreat(inf: float, pop: int):
  tinf = inf
  day = 0
  while tinf > 2.5:
    day += 1
    tinf = retreatOneDay(tinf, pop)
  return day

def daysToExpansion(inf: float, pop: int):
  ret = daysToPendingExpansion(inf,pop)
  ret += 1 # pending day
  ret += 7 # active expansion https://discordapp.com/channels/483005833853009950/483005833853009952/759126832750002186
  return ret

def daysToRetreat(inf: float, pop: int):
  ret = daysToPendingRetreat(inf,pop)
  ret += 1 # pending day
  ret += 6 # active retreat
  ret += 1
  return ret

In [134]:
daysToPendingExpansion(65.9,2200)

2

In [135]:
daysToPendingRetreat(65.9,2200)

15

In [136]:
humanize.naturaldelta(dt.datetime.now() + dt.timedelta(days=0.5 ))

'11 hours'

In [137]:

def getRetreatFrame(inf: float, pop: int):
  dayNum = []
  infs = []
  day = 0

  dayNum.append(day)
  infs.append(inf)

  tinf = inf
  while tinf > 2.5:
    day +=1
    tinf = retreatOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)

  n = 0
  while n<=8:
    day +=1
    tinf = retreatOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)
    n += 1

  df = pd.DataFrame( list(zip(dayNum, infs)), columns=["dayNum","inf"])
  return df

def getAdvanceFrame(inf: float, pop: int, nDays: int):
  dayNum = []
  infs = []
  day = 0

  dayNum.append(day)
  infs.append(inf)

  tinf = inf
  while day<= nDays:
    day +=1
    tinf = expandOneDay(tinf,pop)
    dayNum.append(day)
    infs.append(tinf)

  df = pd.DataFrame( list(zip(dayNum, infs)), columns=["dayNum","inf"])
  return df

In [143]:
inf = 36.0
pop = 36000

nDays = daysToRetreat(inf, pop)
df = getRetreatFrame(inf,pop)
df2 = getAdvanceFrame(inf, pop, nDays)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df2.dayNum, y=df2.inf,
                    mode='lines',
                    name='Ally',
                    marker_color="#0000ff"))
fig.add_trace(go.Scatter(x=df.dayNum, y=df.inf,
                    mode='lines',
                    name='Target',
                    marker_color="#ff0000"))
retLine = list( 2.5 for i in range(df.dayNum.size))
expLine = list( 75.0 for i in range(df.dayNum.size))
fig.add_trace(go.Scatter(x=df2.dayNum, y=retLine,
                    mode='lines',
                    name='Retreat',
                    marker_color="#ff0000",
                    line_dash="dot"))
fig.add_trace(go.Scatter(x=df2.dayNum, y=expLine,
                    mode='lines',
                    name='Expand',
                    marker_color="#00ff00",
                    line_dash="dot"))
fig.show()



In [206]:
def getSimulationFrame(ally_inf: float, targ_inf: float, pop: int):
  #nDays = daysToRetreat(inf, pop)

  dayNum = []
  ally_infs = []
  targ_infs = []
  ally_note = []
  targ_note = []

  day = 0

  dayNum.append(day)
  targ_infs.append(targ_inf)
  ally_infs.append(ally_inf)
  targ_note.append("")
  ally_note.append("")

  tinf = targ_inf
  ainf = ally_inf

  expansion_day = 0
  retreat_day = 0

  while retreat_day<= 7:
    day +=1
    # Save current day's numbers
    painf = ainf
    ptinf = tinf

    # Get next day's numbers
    ainf = expandOneDay(ainf,pop)
    tinf = retreatOneDay(tinf,pop)

    #detect crossing
    if (  (painf<ptinf) and (ainf>=tinf) ):
      minf = (ainf + tinf) / 2.0
      for i in range(5):
          dayNum.append(day)
          targ_infs.append(minf)
          ally_infs.append(minf)
          note = ""
          if i == 0:
            note = "Pending Conflict"
          else:
            note = "Conflict day " + str(i)
          targ_note.append(note)
          ally_note.append(note)
          day += 1
    else: 
      dayNum.append(day)
      note = ""
      if tinf<2.5:
        if retreat_day == 0:
          note = "Pending Retreat"
        else:
          note = "Retreat day " + str(retreat_day)
        retreat_day += 1

      targ_infs.append(tinf)
      targ_note.append(note)

      note = ""
      if ainf>75.0:
        if expansion_day == 0:
          note = "Pending Expansion"
        elif expansion_day<8:
          note = "Expansion day " + str(expansion_day)
        elif expansion_day==8:
          note = "Expansion"
        else:
          note=""
        expansion_day += 1
      ally_infs.append(ainf)
      ally_note.append(note)

  #df = pd.DataFrame( list(zip(dayNum, ally_infs, targ_infs)), columns=["dayNum","ally_inf","target_inf"])
  df = pd.DataFrame( list(zip(dayNum, ally_infs, ally_note, targ_infs, targ_note)), columns=["dayNum","ally_inf","ally_note","target_inf","targ_note"])
  return df

In [207]:
sim = getSimulationFrame( 30, 40, 40000)

In [208]:
ally_color="pink"
target_color = "red"
conflict_color = "yellow"
expansion_color = "magenta"
retreat_color ="deeppink"
  
fig = go.Figure()
fig.add_trace(go.Scatter(x=sim.dayNum, y=sim.ally_inf,
                    mode='lines',
                    name='Ally',
                    marker_color="blue"))
fig.add_trace(go.Scatter(x=sim.dayNum, y=sim.target_inf,
                    mode='lines',
                    name='Target',
                    marker_color="red"))
retLine = list( 2.5 for i in range(sim.dayNum.size))
expLine = list( 75.0 for i in range(sim.dayNum.size))
fig.add_trace(go.Scatter(x=sim.dayNum, y=retLine,
                    mode='lines',
                    name='Retreat',
                    marker_color="#ff0000",
                    line_dash="dot"))
fig.add_trace(go.Scatter(x=sim.dayNum, y=expLine,
                    mode='lines',
                    name='Expand',
                    marker_color="#00ff00",
                    line_dash="dot"))
#fig.update_yaxes(type="log")
fig.show()

In [209]:
sim

,dayNum,ally_inf,ally_note,target_inf,targ_note
0,0,30.000000,,40.000000,
1,1,37.573759,Pending Conflict,37.573759,Pending Conflict
2,2,37.573759,Conflict day 1,37.573759,Conflict day 1
3,3,37.573759,Conflict day 2,37.573759,Conflict day 2
4,4,37.573759,Conflict day 3,37.573759,Conflict day 3
5,5,37.573759,Conflict day 4,37.573759,Conflict day 4
6,7,51.960876,,27.450928,
7,8,60.203617,,22.740790,
8,9,67.032037,,18.838836,
9,10,72.688809,,15.606395,
